Реализуем градиентный спуск для задачи поиска оптимальных коэффициентов в MSE регрессии!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Имеем 1000 объектов и 10 признаков у каждого (+таргет)!

Обучим модель линейной регрессии:

$$
a(x) = \beta_1 d_{1} + \beta_2 d_{2} + \beta_3 d_{3} + \beta_4 d_{4} + \beta_5 d_{5} + \beta_6 d_{6} + \beta_7 d_{7} + \beta_8 d_{8} + \beta_9 d_{9} + \beta_{10} d_{10} + \beta_0
$$

Которая минимизирует MSE:

$$
Q(a(X), Y) = \sum_i^{1000} (a(x_i) - y_i)^2
$$

In [2]:
data = pd.read_csv('data.csv')

data.shape

(1000, 11)

Обучим коэффициенты линейной регрессии с помощью библиотеки <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html"> **sklearn** </a>

Отдельно выведем оценку свободного коэффициента  ($\beta_0$ при $d_0 = 1$)

In [3]:
from sklearn.linear_model import LinearRegression

### Your code is here
model = LinearRegression()
X = data.drop(['target'], axis=1)
Y = data['target']
model.fit(X, Y)

LinearRegression()

In [4]:
for column, coef in zip(X.columns, model.coef_):
    print(column, coef)
print(model.intercept_)

feature_1 0.9999999999999971
feature_2 1.9999999999999978
feature_3 3.0000000000000044
feature_4 4.000000000000004
feature_5 4.999999999999991
feature_6 6.000000000000001
feature_7 6.9999999999999964
feature_8 8.000000000000005
feature_9 8.999999999999996
feature_10 9.999999999999986
4.373157038707131


Теперь вам необходимо реализовать класс для оптимизации коэффициентов линейной регрессии МНК.
Подразумевается, что на вход алгоритм будет принимать следующие параметры:

- 2 pandas датафрейма **samples** и **targets**, содержащих матрицу объектов и ветор ответов соответственно
- значение **learning rate**, который корректирует длину вектора-градиента (чтобы он не взорвался)
- значение **threshold**'а для критерия останова (когда мы считаем, что мы сошлись к оптимуму)
- параметр **copy**, который позволяет либо делать изменения in-place в датафрейме, подающимся в класс, если изменения матрицы объектов в принципе при обучении имеются. Или же копировать объект при инициализации класса и возвращать новый объект, если требуется.

Он будет состоять из следующих важных компонент-методов:

- **add_constant_feature**: добавляет колонку с названием *constant* из единичек к переданному датафрейму **samples**. Это позволяет оценить свободный коэффициент $\beta_0$.

- **calculate_mse_loss**: вычисляет при текущих весах **self.beta** значение среднеквадратической ошибки.

- **calculate_gradient**: вычисляет при текущих весах вектор-градиент по функционалу.

- **iteration**: производит итерацию градиентного спуска, то есть обновляет веса модели, в соответствии с установленным **learning_rate = $\eta$**: $\beta^{(n+1)} = \beta^{(n)} - \eta \cdot \nabla Q(\beta^{(n)})$

- **learn**: производит итерации обучения до того момента, пока не сработает критерий останова обучения. В этот раз критерием останова будет следующее событие: во время крайней итерации изменение в функционале качества модели составило значение меньшее, чем **self.threshold**. Иными словами, $|Q(\beta^{(n)}) - Q(\beta^{(n+1)})| < threshold$.

P.S. установите в **__init__** аттрибут экземпляра с названием **iteration_loss_dict**, который будет устроен следующим образом: на каждой итерации мы будем добавлять в словарь пару ключ-значение, где ключем будет номер итерации $n$, а значением - среднеквадратическая ошибка в точке $\beta^{(n)}$. Это пригодится нам в будущем для визуализации.

### Hint: пример вычисления производной

$$
Q(a, X) = \frac{1}{N}\cdot\sum_{i=1}^N (\beta_1 \cdot d_{i1} + ... + \beta_n \cdot d_{in} - y_i)^2
$$

Выше - минимизируемая функция. Она зависит от n переменных: $\beta_1, ..., \beta_n$. Вектор-градиент - матрица с одной строчкой, состоящей из производных 1го порядка по всем переменным.

$$
\nabla Q(a, X) = (Q'_{\beta_1} \;\;\; Q'_{\beta_2} \;\;\; ... \;\;\; Q'_{\beta_{n-1}}  \;\;\;  Q'_{\beta_n})
$$

Пример вычисления производной по первой переменной:

$$
Q'_{\beta_1} = \frac{2}{N} \cdot \sum_{i=1}^N d_{i1} (\beta_1 \cdot d_{i1} + ... + \beta_{n} \cdot d_{in} - y_i)
$$

Скажем, для нашего датасета X, Y вычислим эту саму производную при начальных единичных коэффициентах $\beta_{start} = (1 \;\;\; 1 \;\;\; ...)$

Получим для каждого объекта в начале выражение из скобочек: 
$$
\beta_1 \cdot d_{i1} + ... + \beta_{n} \cdot d_{in} - y_i
$$

In [5]:
### Инициализируем точку для коэффициентов в виде вектора из единичек
initial_betas = np.ones(X.shape[1])

### Получим выражение выше для каждого объекта. 
### Для этого скалярно перемножим строчки из X на наши beta

scalar_value = np.dot(X, initial_betas.reshape(-1, 1)).ravel()
scalar_value = (scalar_value - Y).values

Теперь полученное значение для каждого объекта умножим на соответствующее значение признака $d_1$:

$$
d_{i1} \cdot (\beta_1 \cdot d_{i1} + ... + \beta_{n} \cdot d_{in} - y_i)
$$

In [6]:
### Возьмем столбик со значениями 1 признака

d_i1 = X.values[:, 0]

### Умножим каждый объект на соответствующее значение признака
scalar_value = scalar_value * d_i1


In [7]:
### Наконец, умножим все на 2 и усреднимся, 
### чтобы получить значение производной по первому параметру

2 * np.mean(scalar_value)

-27.62384887912409

### Эта логика поможем Вам при реализации класса!

learn(self)

метод возвращает итоговую среднеквадратическую ошибку.
метод итеративно вычисляет среднеквадратическую ошибку и вектор-градиент. номер итерации и MSE записываются в словарь *iteration_loss_dict*. критерий останова срабатывает тогда, когда абсолютное значение разницы двух последних MSE меньше *self.threshold*.

In [8]:
class GradientDescentMse:
    """
    Базовый класс для реализации градиентного спуска в задаче линейной МНК регрессии 
    """

    def __init__(self, samples: pd.DataFrame, targets: pd.DataFrame,
                 learning_rate: float = 1e-3, threshold = 1e-6, copy: bool = True):
        """
        :param samples: матрица объектов
        :param targets: вектор (матрица с 1 столбцом) ответов
        :param learning_rate: параметр learning_rate для корректировки нормы градиента
        :param threshold: величина, меньше которой изменение в loss-функции означает 
        :param copy: копировать сэмплы или делать изменения in-place (см. add_constant_feature)
        """
        if copy:
            self.samples = samples.copy()
        else:
            self.samples = samples
        self.targets = targets
        self.learning_rate = learning_rate
        self.threshold = threshold
        
        self.beta = np.ones(self.samples.shape[1])
        self.iteration_num = 0
        self.iteration_loss_dict = {}
        
    def add_constant_feature(self):
        """
        Метод для создания константной фичи в матрице объектов samples
        """
        self.samples['constant'] = 1    
        self.beta = np.append(self.beta, 1)
        
    def calculate_mse_loss(self) -> float:
        """
        Метод для создания константной фичи в матрице объектов samples
        
        :return: среднеквадратическая ошибка при текущих весах модели : float
        """
        loss = np.dot(self.samples, self.beta)  - self.targets.values
        
        return np.mean(loss**2)

    def calculate_gradient(self) -> np.ndarray:
        """
        Метод для вычисления вектора-градиента
        
        :return: вектор-градиент, т.е. массив, содержащий соответствующее количество производных по каждой переменной : np.ndarray
        """
        shift = np.dot(self.samples, self.beta)  - self.targets.values
        derivatives = 2 * np.dot(shift, self.samples) / self.samples.shape[0]
        
        return derivatives
    
    def iteration(self):
        """
        Обновляем веса модели в соответствии с текущим вектором-градиентом
        """
        self.beta = self.beta - self.learning_rate * self.calculate_gradient()
        
    def learn(self):
        """
        Итеративное обучение весов модели до срабатывания критерия останова
        """
        previous_mse = self.calculate_mse_loss()
        
        self.iteration()
        
        next_mse = self.calculate_mse_loss()
        
        self.iteration_loss_dict[0] = previous_mse
        self.iteration_loss_dict[1] = next_mse
        
        self.iteration_num = 1
        
        while abs(next_mse - previous_mse) >= self.threshold:
            
            previous_mse = next_mse
            
            self.iteration()
            
            next_mse = self.calculate_mse_loss()
            
            self.iteration_loss_dict[self.iteration_num+1] = next_mse
            
            self.iteration_num += 1

Обучим коэффициенты линейной модели с помощью реализованного нами градиентного спуска, не забыв добавить свободную переменную. Получились ли такие же коэффициенты, как и при использовании **LinearRegression** из **sklearn**? Если нет, то почему они отличаются, на Ваш взгляд, и сильно ли?

In [9]:
GD = GradientDescentMse(samples=X, targets=Y)
GD.add_constant_feature()
GD.learn()

In [10]:
GD.iteration_loss_dict

{0: 682.668858746845,
 1: 673.1874227973016,
 2: 663.839092118565,
 3: 654.6219976377024,
 4: 645.534296527588,
 5: 636.574171838355,
 6: 627.7398321340238,
 7: 619.0295111342318,
 8: 610.4414673609944,
 9: 601.9739837904258,
 10: 593.6253675093516,
 11: 585.3939493767411,
 12: 577.2780836898975,
 13: 569.2761478553298,
 14: 561.386542064253,
 15: 553.6076889726401,
 16: 545.9380333857687,
 17: 538.3760419471971,
 18: 530.9202028321077,
 19: 523.5690254449574,
 20: 516.3210401213715,
 21: 509.17479783422556,
 22: 502.12886990385465,
 23: 495.18184771233024,
 24: 488.3323424217506,
 25: 481.5789846964864,
 26: 474.9204244293271,
 27: 468.3553304714725,
 28: 461.88239036631506,
 29: 455.50031008696254,
 30: 449.20781377744396,
 31: 443.0036434975512,
 32: 436.88655897126324,
 33: 430.8553373387024,
 34: 424.9087729115747,
 35: 419.0456769320434,
 36: 413.26487733498936,
 37: 407.5652185136088,
 38: 401.94556108830346,
 39: 396.4047816788155,
 40: 390.94177267956235,
 41: 385.555442038126

In [11]:
print('Веса модели при переменных d0, d1, ..., d10 равны соответственно: \n\n' + str(GD.beta))

Веса модели при переменных d0, d1, ..., d10 равны соответственно: 

[0.88733305 1.90100713 2.88063607 3.87662612 4.89623507 5.89126182
 6.89254811 7.90311947 8.87580109 9.86109585 4.94854733]


Попробуйте теперь изменить значения **learning_rate** и/или **threshold**. Например, установите длину шага $\eta = 1$. Что произошло и почему такое возможно?

In [12]:
### Your code is here
GD_new = GradientDescentMse(samples=X, targets=Y, learning_rate=1)
GD_new.add_constant_feature()
GD_new.learn()

/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/tmp/ipykernel_220/4100436744.py:76: RuntimeWarning: invalid value encountered in double_scalars
  while abs(next_mse - previous_mse) >= self.threshold:


In [13]:
print('Веса модели при переменных d0, d1, ..., d10, d0 равны соответственно: \n\n' + str(GD_new.beta))

Веса модели при переменных d0, d1, ..., d10, d0 равны соответственно: 

[9.40188501e+152 9.18141651e+152 8.60666268e+152 8.65906169e+152
 8.84957221e+152 9.26558544e+152 9.00779348e+152 9.33707984e+152
 8.88112792e+152 8.93870956e+152 1.77388075e+153]


В машинном обучении зачастую исследуют так называемые **траектории обучения** (или **learning paths**). Это графики, показывающие, как во время обучения при каждой следующей итерации изменялось значение минимизируемого функционала. Постройте такие траектории для различных **learning rate**'ов и **threshold**'ов. Советуем использовать для этого разобранный на занятиях **add_subplot** метод. 

Возьмите следующие **threshold**'ы: 1e-2, 1e-3, 1e-4, 1e-5

И следующие значения **learning rate**'а: 1e-1, 5e-2, 1e-2, 5e-3, 1e-3

У вас должен получиться примерно такой график (см. ниже, значения среднеквадратической ошибки мы намеренно замазали оранжевыми квадратиками, чтобы не спойлерить вам результаты).

Как и подобает хорошим Data Scientist'ам, не забывайте подписывать графики, оси, а так же делать элементы ваших визуализаций читаемыми и видимыми. Советуем пересмотреть методы и параметры форматирования из лекции.

При какой комбинации **threshold** - **learning rate** из возможных предложенных выше, получается достигнуть меньшего значения нашей минимизируемой функции? Запишите каждой из значений в легенде на графиках.


In [ ]:
### Your code is here
fig = plt.figure()

fig.set_size_inches(13, 10)

rates = [0.1, 0.05, 0.01, 0.005, 0.001]
thresholds = [0.01, 0.001, 0.0001, 0.00001]

for i in range(len(thresholds)):
    thresh = thresholds[i]
    
    ax_ = fig.add_subplot(2, 2, i+1)
    
    Q_values = []
    
    for lr in rates:
        GD = GradientDescentMse(samples=X, targets=Y, 
                                learning_rate=lr, threshold=thresh)
        GD.add_constant_feature()
        GD.learn()

        learning_path = GD.iteration_loss_dict

        plt.plot(learning_path.keys(), learning_path.values())
        plt.title(f'Threshold = {thresh}')
        plt.ylim(0, 100)
        plt.xlim(0, 2000)
        
        Q_values.append(str(round(list(learning_path.values())[-1], ndigits=4)))
    
    plt.ylabel('Среднеквадратическая ошибка')
    plt.xlabel('Номер итерации')
    plt.legend([f'Learning rate equals to {rates[i]}' + ' with Q = ' + Q_values[i] for i in range(len(rates))])

fig.tight_layout() 

plt.show()